[View in Colaboratory](https://colab.research.google.com/github/nailbrainz/tensorflow/blob/master/mnist_feedfoward_cnn.ipynb)

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
print("packs loaded")


packs loaded


In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
randidx = np.random.randint(testimg.shape[0], size=150)
print(type(testimg))
print(testimg.shape)
testimg = testimg[randidx, :]
testlabel = testlabel[randidx, :]
print(type(testimg))
print(testimg.shape)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
<class 'numpy.ndarray'>
(10000, 784)
<class 'numpy.ndarray'>
(150, 784)


In [0]:
device_type = "/gpu:0"

In [4]:
n_input = trainimg.shape[1]
print(trainimg.shape)
n_output = trainlabel.shape[1]
_input_r = tf.reshape(trainimg, shape=[-1, 28, 28, 1])
print("conv between input:" + str(_input_r.shape) + " and kernel:" + "[3, 3, 1, 64]")

with tf.device(device_type):
    weights = {
        'wc1':tf.Variable(tf.truncated_normal([3, 3, 1, 64], stddev=0.1)),
        'wc2':tf.Variable(tf.truncated_normal([3, 3, 64, 128], stddev=0.1)),
        'wd1':tf.Variable(tf.truncated_normal([7*7*128, 1024], stddev=0.1)),
        'wd2':tf.Variable(tf.truncated_normal([1024, n_output], stddev=0.1))
    }
    biases = {
        'bc1':tf.Variable(tf.random_normal([64], stddev=0.1)),
        'bc2':tf.Variable(tf.random_normal([128], stddev=0.1)),
        'bd1':tf.Variable(tf.random_normal([1024], stddev=0.1)),
                'bd2':tf.Variable(tf.random_normal([n_output], stddev=0.1))
    }
    def conv_basic(_input, _w, _b, _keepratio):
        #input
        _input_r = tf.reshape(_input, shape=[-1, 28, 28, 1])
        
        #conv layer 1
        _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1 for x in range(4)], padding="SAME")
        _mean, _var = tf.nn.moments(_conv1, [0, 1, 2])
        _conv1 = tf.nn.batch_normalization(_conv1, _mean, _var, 0, 1, 0.0001)
        _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1, _b['bc1']))
        _pool1 = tf.nn.max_pool(_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
        
        #conv layer 2
        _conv2 = tf.nn.conv2d(_pool_dr1, _w['wc2'], strides=[1,1,1,1], padding="SAME")
        _mean, _var = tf.nn.moments(_conv2, [0, 1, 2])
        _conv2 = tf.nn.batch_normalization(_conv2, _mean, _var, 0, 1, 0.0001)
        _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2, _b['bc2']))
        _pool2 = tf.nn.max_pool(_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
        
        #vectorize
        _dense1 = tf.reshape(_pool_dr2, [-1, _w['wd1'].get_shape().as_list()[0]])
        
        #FCN1_poor_dr1
        _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
        _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
        
        #FCN2
        _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
        
        out = { 'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1, 'pool1_dr1': _pool_dr1,
            'conv2': _conv2, 'pool2': _pool2, 'pool_dr2': _pool_dr2, 'dense1': _dense1,
            'fc1': _fc1, 'fc_dr1': _fc_dr1, 'out': _out
        }
        return out
print ("CNN READY")

(55000, 784)
conv between input:(55000, 28, 28, 1) and kernel:[3, 3, 1, 64]
CNN READY


In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

with tf.device(device_type):
    _pred = conv_basic(x, weights, biases, keepratio)['out']
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=_pred))
    optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    _corr = tf.equal(tf.argmax(y, 1), tf.argmax(_pred, 1))
    accr = tf.reduce_mean(tf.cast(_corr, tf.float32))
    init = tf.initialize_all_variables()
    
#saver??
save_step = 1
saver = tf.train.Saver(max_to_keep=3)
print("graph rdy")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
graph rdy


In [0]:
do_train = 1
_config = tf.ConfigProto(allow_soft_placement=True)
sess = tf.Session(config=_config)
sess.run(init)

In [7]:
training_epochs = 15
batch_size = 50
display_step = 1
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_gpus())

if do_train:
    for epoch in range(training_epochs):
        print("start")
        avg_cost = 1
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            fds = {x:batch_xs, y:batch_ys, keepratio:0.7}
            sess.run(optm, feed_dict=fds)
            avg_cost += sess.run(cost, feed_dict=fds)/total_batch
        print('end')
        if epoch % display_step == 0:
            print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
            train_acc = sess.run(accr, feed_dict={x:batch_xs, y:batch_ys, keepratio:1.})
            print (" Training accuracy: %.3f" % (train_acc))
            test_acc = sess.run(accr, feed_dict={x:testimg, y:testlabel, keepratio:1.})
            print (" Test accuracy: %.3f" % (test_acc))

        # Save Net
        if epoch % save_step == 0:
            saver.save(sess, "nets/cnn_mnist_basic.ckpt-" + str(epoch))

    print ("OPTIMIZATION FINISHED")


['/device:CPU:0', '/device:GPU:0']
start
end
Epoch: 000/015 cost: 1.993025251
 Training accuracy: 0.980
 Test accuracy: 0.940
start
end
Epoch: 001/015 cost: 1.176778478
 Training accuracy: 0.980
 Test accuracy: 0.960
start
end
Epoch: 002/015 cost: 1.121932805
 Training accuracy: 1.000
 Test accuracy: 0.980
start
end
Epoch: 003/015 cost: 1.105492078
 Training accuracy: 1.000
 Test accuracy: 0.987
start
end
Epoch: 004/015 cost: 1.083105101
 Training accuracy: 1.000
 Test accuracy: 0.987
start
end
Epoch: 005/015 cost: 1.070979459
 Training accuracy: 1.000
 Test accuracy: 0.987
start
end
Epoch: 006/015 cost: 1.059572229
 Training accuracy: 1.000
 Test accuracy: 0.987
start
end
Epoch: 007/015 cost: 1.054356091
 Training accuracy: 0.980
 Test accuracy: 0.987
start
end
Epoch: 008/015 cost: 1.044095727
 Training accuracy: 1.000
 Test accuracy: 0.980
start
end
Epoch: 009/015 cost: 1.038433382
 Training accuracy: 1.000
 Test accuracy: 0.987
start
end
Epoch: 010/015 cost: 1.033188634
 Training ac